In [2]:
import os
import uvicorn
import requests
import nest_asyncio
from fastapi import FastAPI, HTTPException, BackgroundTasks
from pydantic import BaseModel, Field
from supabase import create_client, Client
from typing import Optional, Dict, Any
from dotenv import load_dotenv

# ==========================================
# Configuration
# ==========================================



load_dotenv()

nest_asyncio.apply()

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
MAKE_WEBHOOK_URL = os.getenv("MAKE_WEBHOOK_URL")

if not SUPABASE_URL or not SUPABASE_KEY:
    raise ValueError("SUPABASE_URL and SUPABASE_KEY must be set")

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
app = FastAPI(title="Pharmacy Stock API")

# ==========================================
# Models
# ==========================================

class StockCheckRequest(BaseModel):
    med_code: str = Field(..., description="Medication code")
    needed_quantity: int = Field(..., description="Quantity needed", gt=0)
    alternative_med_code: Optional[str] = Field(None, description="Alternative medication code")
    reorder_threshold_override: Optional[int] = Field(None, description="Override reorder threshold")

class StockResponse(BaseModel):
    stockStatus: str
    quantityOnHand: int
    neededQuantity: int
    sufficientStock: bool
    medicationName: str
    medCode: str
    supplierName: str
    supplierEmail: str
    message: Optional[str] = None

class DispenseRequest(BaseModel):
    medication_name: str
    quantity: int = Field(..., gt=0)

class RestockRequest(BaseModel):
    reorder_id: int
    quantity_received: int = Field(..., gt=0)

# ==========================================
# Database Operations
# ==========================================

def get_medication(med_code: str) -> Optional[Dict[str, Any]]:
    if not med_code:
        return None
    
    response = (
        supabase.table("pharmacy_inventory")
        .select("medication_id, supplier_id, medication_name, quantity_on_hand, reorder_level, med_code, suppliers(supplier_name, contact_email)")
        .eq("med_code", med_code)
        .limit(1)
        .execute()
    )
    return response.data[0] if response.data else None

def get_supplier_info(inventory_record: Dict) -> Dict[str, str]:
    supplier = inventory_record.get("suppliers", {})
    return {
        "name": supplier.get("supplier_name", "Unknown Supplier"),
        "email": supplier.get("contact_email", "")
    }

def has_sufficient_stock(medication: Dict, needed_qty: int) -> bool:
    return medication and medication.get("quantity_on_hand", 0) >= needed_qty

def update_stock_quantity(medication_name: str, quantity_change: int) -> Optional[Dict[str, Any]]:
    """
    Updates stock via RPC and returns the full updated record.
    Returns None if the medication was not found or an error occurred.
    """
    try:
        response = supabase.rpc("update_inventory_count", {
            "p_medication_name": medication_name, 
            "p_quantity_change": quantity_change
        }).execute()
        
        # If no rows were found/updated, data will be None
        if not response.data:
            print(f"Warning: Medication '{medication_name}' not found during update.")
            return None

        return response.data

    except Exception as e:
        print(f"Database Error: {e}")
        return None
def has_active_reorder(medication_id: int) -> bool:
    response = (
        supabase.table("reorder_history")
        .select("reorder_id")
        .eq("medication_id", medication_id)
        .neq("status", "Received") 
        .neq("status", "Cancelled") 
        .execute()
    )
    return len(response.data) > 0

def log_reorder_event(medication: Dict, quantity: int):
    try:
        data = {
            "medication_id": medication["medication_id"],
            "supplier_id": medication["supplier_id"],
            "quantity_ordered": quantity,
            "status": "Ordered",
            "reorder_date": "now()"
        }
        supabase.table("reorder_history").insert(data).execute()
        print(f"🔒 Reorder locked for {medication['medication_name']}")
    except Exception as e:
        print(f"Failed to log reorder: {e}")

def mark_order_received(reorder_id: int, qty_received: int):
    """
    Updates reorder status and increases inventory via RPC.
    """
    # 1. Get details from the reorder record
    reorder_res = (
        supabase.table("reorder_history")
        .select("medication_id, status")
        .eq("reorder_id", reorder_id)
        .single()
        .execute()
    )
    
    if not reorder_res.data:
        raise ValueError("Reorder ID not found")
        
    reorder_data = reorder_res.data
    
    if reorder_data["status"] == "Received":
        raise ValueError("This order has already been processed")

    # 2. Get the med_code (needed for the RPC)
    med_res = (
        supabase.table("pharmacy_inventory")
        .select("med_code")
        .eq("medication_id", reorder_data["medication_id"])
        .single()
        .execute()
    )
    
    if not med_res.data:
        raise ValueError("Associated medication not found")
        
    med_code = med_res.data["med_code"]

    # 3. Update stock (Positive number for adding stock)
    update_stock_quantity(medication_name, qty_received)

    # 4. Mark history as received
    supabase.table("reorder_history").update({
        "status": "Received",
        "actual_delivery_date": "now()",
        "quantity_ordered": qty_received
    }).eq("reorder_id", reorder_id).execute()

    print(f"✅ Order {reorder_id} received. Stock increased.")

# ==========================================
# Reorder Logic
# ==========================================

def send_reorder_alert(medication: Dict, threshold: int):
    if not MAKE_WEBHOOK_URL:
        return
    
    supplier = get_supplier_info(medication)
    payload = {
        "type": "Low Stock Alert",
        "medicationName": medication["medication_name"],
        "medCode": medication["med_code"],
        "currentStock": medication["quantity_on_hand"],
        "reorderLevel": threshold,
        "supplierName": supplier["name"],
        "supplierEmail": supplier["email"],
        "status": "Urgent"
    }
    
    try:
        requests.post(MAKE_WEBHOOK_URL, json=payload, timeout=5)
        print(f"Reorder alert sent: {payload['medicationName']}")
    except Exception as e:
        print(f"Webhook failed: {e}")

async def check_reorder_needed(med_code: str, threshold_override: Optional[int] = None):
    medication = get_medication(med_code)
    if not medication:
        return
    
    current_qty = medication["quantity_on_hand"]
    db_threshold = medication.get("reorder_level", 5)
    threshold = threshold_override if threshold_override is not None else db_threshold
    
    if current_qty <= threshold:
        if has_active_reorder(medication["medication_id"]):
            print(f"Check skipped: Active order exists for {medication['medication_name']}")
            return

        send_reorder_alert(medication, threshold)
        log_reorder_event(medication, quantity=50)

# ==========================================
# API Endpoints
# ==========================================

@app.get("/")
def health_check():
    return {"status": "active", "version": "1.0"}

@app.post("/check-stock", response_model=StockResponse)
async def check_stock(body: StockCheckRequest, background_tasks: BackgroundTasks):
    background_tasks.add_task(check_reorder_needed, body.med_code, body.reorder_threshold_override)
    if body.alternative_med_code:
        background_tasks.add_task(check_reorder_needed, body.alternative_med_code, body.reorder_threshold_override)
    
    primary = get_medication(body.med_code)
    if not primary:
        raise HTTPException(status_code=404, detail=f"Medication {body.med_code} not found")
    
    if has_sufficient_stock(primary, body.needed_quantity):
        supplier = get_supplier_info(primary)
        return StockResponse(
            stockStatus="OK",
            quantityOnHand=primary["quantity_on_hand"],
            neededQuantity=body.needed_quantity,
            sufficientStock=True,
            medicationName=primary["medication_name"],
            medCode=primary["med_code"],
            supplierName=supplier["name"],
            supplierEmail=supplier["email"]
        )
    
    if body.alternative_med_code:
        alternative = get_medication(body.alternative_med_code)
        if has_sufficient_stock(alternative, body.needed_quantity):
            supplier = get_supplier_info(alternative)
            return StockResponse(
                stockStatus="Alternative Available",
                quantityOnHand=alternative["quantity_on_hand"],
                neededQuantity=body.needed_quantity,
                sufficientStock=True,
                medicationName=alternative["medication_name"],
                medCode=alternative["med_code"],
                supplierName=supplier["name"],
                supplierEmail=supplier["email"],
                message=f"Primary stock insufficient ({primary['quantity_on_hand']} available). Using alternative."
            )
    
    return StockResponse(
        stockStatus="Insufficient Stock",
        quantityOnHand=primary["quantity_on_hand"],
        neededQuantity=body.needed_quantity,
        sufficientStock=False,
        medicationName=primary["medication_name"],
        medCode=primary["med_code"],
        supplierName="N/A",
        supplierEmail="",
        message=f"Insufficient stock. Need {body.needed_quantity}, have {primary['quantity_on_hand']}."
    )

@app.post("/dispense")
async def dispense_medication(body: DispenseRequest, background_tasks: BackgroundTasks):
    # 1. Attempt to update stock (passing negative quantity to subtract)
    updated_med = update_stock_quantity(body.medication_name, -body.quantity)
    
    # 2. Handle "Not Found" error
    if not updated_med:
        raise HTTPException(
            status_code=404, 
            detail=f"Medication '{body.medication_name}' not found"
        )

    # 3. Extract necessary data from the DB result
    med_code = updated_med.get("med_code")
    new_qty = updated_med.get("quantity_on_hand")

    # 4. Trigger background reorder check
    if med_code:
        background_tasks.add_task(check_reorder_needed, med_code)

    # 5. Return success response
    return {
        "status": "Dispensed",
        "medication": body.medication_name,
        "med_code": med_code,
        "quantity_dispensed": body.quantity,
        "new_quantity_on_hand": new_qty
    }


@app.post("/receive-shipment")
async def receive_shipment(body: RestockRequest):
    try:
        # We cleaned this up to not need med_id from the body
        mark_order_received(
            reorder_id=body.reorder_id, 
            qty_received=body.quantity_received
        )

        return {
            "status": "Stock Updated", 
            "message": "Order marked as Received and inventory updated."
        }
    except ValueError as e:
        raise HTTPException(status_code=400, detail=str(e))

# ==========================================
# Server
# ==========================================

if __name__ == "__main__":
    print("🚀 Pharmacy API starting on port 8080...")
    config = uvicorn.Config(app, host="0.0.0.0", port=8080, log_level="info")
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [1197]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
🚀 Pharmacy API starting on port 8080...
INFO:     10.236.52.217:35862 - "POST /dispense HTTP/1.1" 200 OK
✋ Check skipped: Active order exists for Ibuprofen
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1197]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4337a8e3-98a6-4061-a570-0c3c734ce2f0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>